In [1]:
from __future__ import print_function, division
import sys
sys.path.append("../")

In [2]:
from dsbtorch import IterableVideoSlidingWindowDataset, VideoSlidingWindowDataset, scan_dataset

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pathlib
import pickle
import tqdm
from torch.utils.tensorboard import SummaryWriter

plt.ion()   # interactive mode

In [4]:
device = torch.device("cuda")
torchvision.set_image_backend('accimage')

In [5]:
t = transforms.Compose([
    transforms.Resize((144, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
data_dir = "/home/ubuntu/data/dataset_511/"

In [7]:
dataset_names = ['dev', 'test', 'train']

In [8]:
scanned_datasets = {}
for x in dataset_names:
    sddir = pathlib.Path("scans/" + os.path.join(data_dir, x))
    sdfile = sddir / "ws1-psr1-nsr1.pkl"
    print(sdfile)
    if not sdfile.exists():
        raise ValueError("You need to use the ScanDatasets notebook first to scan & pickle the dataset.")
    with open(sdfile, 'rb') as f:
        scanned_datasets[x] = pickle.load(f)

scans/home/ubuntu/data/dataset_511/dev/ws1-psr1-nsr1.pkl
scans/home/ubuntu/data/dataset_511/test/ws1-psr1-nsr1.pkl


In [9]:
datasets = {x: VideoSlidingWindowDataset(scanned_datasets[x], t) for x in dataset_names}

In [10]:
output_dir = "/home/ubuntu/data/encoded_dataset_511/"

In [11]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model.load_state_dict(torch.load("/home/ubuntu/data/DeepSponsorBlock/results/attempt2-resnet50-sr10-sgd-lr10-decay1.weights"))

<All keys matched successfully>

In [12]:
# Remove the FC layer.
model.fc = nn.Identity()

for param in model.parameters():
    param.requires_grad = False

In [13]:
model = model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
batch_size = 1024
dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=batch_size, num_workers=6, pin_memory=True) for x in dataset_names}
dataset_sizes = {x: len(datasets[x]) for x in dataset_names}

In [18]:
for x in dataset_names:
    sd = scanned_datasets[x]

    out_files = [output_dir / dir_list[0].parent.relative_to(data_dir) for dir_list in sd.cumulative_dirs]
    lengths = list(np.diff(np.array(sd.cumulative_indices + [sd.n_indices])))
    
    # Reverse them to use as a stack.
    out_files.reverse()
    lengths.reverse()
    
    encoder_outputs = []
    acc_labels = []
    for imgs, lbls in tqdm.tqdm(dataloaders[x]):
        imgs = torch.reshape(imgs, (-1, 3, 144, 256)).to(device)
        encoder_outputs.append(model(imgs))
        acc_labels.append(torch.reshape(lbls, (-1, )))

        while lengths and sum(x.shape[0] for x in encoder_outputs) >= lengths[-1]:
            out_path = out_files.pop()
            out_path.parent.mkdir(parents=True, exist_ok=True)

            combined_encoder_outputs = torch.cat(encoder_outputs)
            combined_labels = torch.cat(acc_labels)

            length = lengths.pop()

            encoder_outputs = [combined_encoder_outputs[length:]]
            acc_labels = [combined_labels[length:]]

            cnn_outputs, labels = (combined_encoder_outputs[:length], combined_labels[:length])
            np.save(out_path.with_suffix('.emb'), cnn_outputs.cpu().numpy())
            np.save(out_path.with_suffix('.lbl'), labels.cpu().numpy())



  0%|          | 0/1126 [00:00<?, ?it/s]

  0%|          | 1/1126 [00:04<1:22:16,  4.39s/it]

  0%|          | 2/1126 [00:09<1:23:27,  4.46s/it]

  0%|          | 3/1126 [00:13<1:23:50,  4.48s/it]

  0%|          | 4/1126 [00:22<1:50:11,  5.89s/it]

  1%|          | 6/1126 [00:32<1:43:05,  5.52s/it]

  1%|          | 7/1126 [00:36<1:38:21,  5.27s/it]

  1%|          | 9/1126 [00:41<1:21:47,  4.39s/it]

  1%|          | 10/1126 [00:46<1:23:02,  4.47s/it]

  1%|          | 11/1126 [00:55<1:49:50,  5.91s/it]

  1%|          | 12/1126 [01:00<1:42:53,  5.54s/it]

  1%|          | 13/1126 [01:04<1:38:18,  5.30s/it]

  1%|          | 14/1126 [01:09<1:35:00,  5.13s/it]

  1%|▏         | 16/1126 [01:14<1:19:29,  4.30s/it]

  2%|▏         | 17/1126 [01:18<1:21:26,  4.41s/it]

  2%|▏         | 18/1126 [01:23<1:22:52,  4.49s/it]

  2%|▏         | 19/1126 [01:28<1:23:48,  4.54s/it]

  2%|▏         | 20/1126 [01:32<1:24:28,  4.58s/it]

  2%|▏         | 21/1126 [01:37<1:24:55,  4.61s/it]

  2%|▏   

KeyboardInterrupt: 

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
